Reference:

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Perceptron.html
https://scikit-learn.org/stable/modules/sgd.html
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html
https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html
https://github.com/leelaylay/TweetSemEval
https://towardsdatascience.com/yet-another-twitter-sentiment-analysis-part-1-tackling-class-imbalance-4d7a7f717d44
https://github.com/cbaziotis/datastories-semeval2017-task4


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
pip install emoji


     |████████████████████████████████| 170 kB 5.0 MB/s 
  Created wheel for emoji: filename=emoji-1.6.1-py3-none-any.whl size=169313 sha256=ff65e34af1c5d42f34a5ef72488a2118122b0bb04a6231220019854553fa8c2d
  Stored in directory: /root/.cache/pip/wheels/ea/5f/d3/03d313ddb3c2a1a427bb4690f1621eea60fe6f2a30cc95940f
Successfully built emoji


In [ ]:
pip install unidecode

     |████████████████████████████████| 235 kB 5.0 MB/s 


In [ ]:
import glob
all_files = glob.glob("twitter*.tsv")
frame = pd.DataFrame()
list_ = []
for file_ in all_files:
    df = pd.read_csv(file_,index_col=None, sep='\t', header=None, names=['id','sentiment','text','to_delete'])
    list_.append(df.iloc[:,:-1])
df = pd.concat(list_)
df = df.drop_duplicates()
df = df.reset_index(drop=True)
df.tail()

,id,sentiment,text
41700,103158179306807296,positive,RT @MNFootNg It's monday and Monday Night Foot...
41701,103157324096618497,positive,All I know is the road for that Lomardi start ...
41702,100259220338905089,neutral,"All Blue and White fam, we r meeting at Golden..."
41703,104230318525001729,positive,@DariusButler28 Have a great game agaist Tam...
41704,100461938533863424,negative,I'm pisseeedddd that I missed Kid Cudi's show ...


Therea are 41705  tweets in the dataset

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41705 entries, 0 to 41704
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         41705 non-null  int64 
 1   sentiment  41705 non-null  object
 2   text       41705 non-null  object
dtypes: int64(1), object(2)
memory usage: 977.6+ KB


In [ ]:
df['token_length'] = [len(x.split(" ")) for x in df.text]
max(df.token_length)

53

In [ ]:
df.loc[df.token_length.idxmax(),'text']


'NO. 108 UN JORIT MAR DALA ZAYUN                                  (Watchman on the Walls of Zion) Doh is C  1. Un... http://t.co/gNu3AzZ6qH'

In [ ]:
df.sentiment.value_counts()

neutral     19466
positive    15754
negative     6485
Name: sentiment, dtype: int64

# Data Cleaning

## Spell Correlction 
it is a simple spell correction dealing with repeated characters such as “sooooo goooood”. If the same character is repeated more than two times, it shortens the repetition to two. For example “sooooo goooood” will be transformed as “soo good”. This is not a perfect solution since even after correction, in case of “soo”, it is not a correct spelling. But at least it will help to reduce feature space by making “sooo”, “soooo”, “sooooo” to the same word “soo”

In [ ]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", 
                   "can't've": "cannot have", "'cause": "because", "could've": "could have", 
                   "couldn't": "could not", "couldn't've": "could not have","didn't": "did not", 
                   "doesn't": "does not", "don't": "do not", "hadn't": "had not", 
                   "hadn't've": "had not have", "hasn't": "has not", "haven't": "have not", 
                   "he'd": "he would", "he'd've": "he would have", "he'll": "he will", 
                   "he'll've": "he will have", "he's": "he is", "how'd": "how did", 
                   "how'd'y": "how do you", "how'll": "how will", "how's": "how is", 
                   "I'd": "I would", "I'd've": "I would have", "I'll": "I will", 
                   "I'll've": "I will have","I'm": "I am", "I've": "I have", 
                   "i'd": "i would", "i'd've": "i would have", "i'll": "i will", 
                   "i'll've": "i will have","i'm": "i am", "i've": "i have", 
                   "isn't": "is not", "it'd": "it would", "it'd've": "it would have", 
                   "it'll": "it will", "it'll've": "it will have","it's": "it is", 
                   "let's": "let us", "ma'am": "madam", "mayn't": "may not", 
                   "might've": "might have","mightn't": "might not","mightn't've": "might not have", 
                   "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", 
                   "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", 
                   "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not",
                   "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", 
                   "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", 
                   "she's": "she is", "should've": "should have", "shouldn't": "should not", 
                   "shouldn't've": "should not have", "so've": "so have","so's": "so as", 
                   "this's": "this is",
                   "that'd": "that would", "that'd've": "that would have","that's": "that is", 
                   "there'd": "there would", "there'd've": "there would have","there's": "there is", 
                       "here's": "here is",
                   "they'd": "they would", "they'd've": "they would have", "they'll": "they will", 
                   "they'll've": "they will have", "they're": "they are", "they've": "they have", 
                   "to've": "to have", "wasn't": "was not", "we'd": "we would", 
                   "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", 
                   "we're": "we are", "we've": "we have", "weren't": "were not", 
                   "what'll": "what will", "what'll've": "what will have", "what're": "what are", 
                   "what's": "what is", "what've": "what have", "when's": "when is", 
                   "when've": "when have", "where'd": "where did", "where's": "where is", 
                   "where've": "where have", "who'll": "who will", "who'll've": "who will have", 
                   "who's": "who is", "who've": "who have", "why's": "why is", 
                   "why've": "why have", "will've": "will have", "won't": "will not", 
                   "won't've": "will not have", "would've": "would have", "wouldn't": "would not", 
                   "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would",
                   "y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                   "you'd": "you would", "you'd've": "you would have", "you'll": "you will", 
                   "you'll've": "you will have", "you're": "you are", "you've": "you have" } 

In [ ]:
import codecs
import unidecode
import re
import spacy
nlp = spacy.load("en_core_web_sm")


def spacy_cleaner(text):
    try:
        decoded = unidecode.unidecode(codecs.decode(text, "unicode_escape"))
    except:
        decoded = unidecode.unidecode(text)
    apostrophe_handled = re.sub("’", "'", decoded)
    expanded = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in apostrophe_handled.split(" ")])
    parsed = nlp(expanded)
    final_tokens = []
    for t in parsed:
        if t.is_punct or t.is_space or t.like_num or t.like_url or str(t).startswith('@'):
            pass
        else:
            if t.lemma_ == '-PRON-':
                final_tokens.append(str(t))
            else:
                sc_removed = re.sub("[^a-zA-Z]", '', str(t.lemma_))
                if len(sc_removed) > 1:
                    final_tokens.append(sc_removed)
    joined = ' '.join(final_tokens)
    spell_corrected = re.sub(r'(.)\1+', r'\1\1', joined)
    return spell_corrected

In [ ]:
pd.set_option('display.max_colwidth', None)
df.text[:10]

0                                      Saturday without Leeds United is like Sunday without a Sunday dinner it doesn't feel normal at all (Ryan)
1                                                   Catch Rainbow Valley at the @CBC #IMAF2014 Gala. Oct 26 @TheGuildPEI! http://t.co/S5aJAEpnkY
2    @NiklaklePinkel it doesn't really count, I was decorating a haunted house for a Halloween program tomorrow...I get paid lots of overtime:-)
3         #BEARDOWN Wish us luck...we may need it. (@ Georgia Dome for Atlanta Falcons vs. Chicago Bears in Atlanta, GA) https://t.co/D0nAGYO4nO
4         We're so excited to be part of the Still We Rise Gala on Dec 3. Join us! @warriors4peace http://t.co/MsKuNg5VMH http://t.co/BdQm1uZe4J
5                             Not really feeling this new season of American Horror Story. 1st 2 season were legit, it went downhill after that.
6                                   @jag_meister1980 Dude, c'mon. Bears are going to the Georgia Dome. That's the one place the Fa

In [ ]:
#Applied the custom cleaner
[spacy_cleaner(t) for t in df.text[:10]]


['Saturday without Leeds United be like Sunday without Sunday dinner it do not feel normal at all Ryan',
 'Catch Rainbow Valley at the IMAF Gala Oct',
 'it do not really count I be decorate haunt house for Halloween program tomorrow I get pay lot of overtime',
 'BEARDOWN wish us luck we may need it Georgia Dome for Atlanta Falcons vs Chicago Bears in Atlanta GA',
 'We be so excited to be part of the still We rise Gala on Dec join us',
 'not really feel this new season of American Horror Story st season be legit it go downhill after that',
 'Dude cmon bear be go to the Georgia Dome that be the place the Falcons CAN win',
 'I wonder if Billy Joe Armstrong Green Day get wake up every October st that joke would be funny the first time but prob get annoying',
 'late upload from the Bears game Sunday the Georgia Dome',
 'I catch up on AHS I be have to do scandal and how to get away with murder tomorrow']

In [ ]:
#Apply the cleaner to all dataset
df['clean_text'] = [spacy_cleaner(t) for t in df.text]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: invalid escape sequence '\o'
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: invalid escape sequence '\('
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: invalid escape sequence '\ '
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: invalid escape sequence '\i'
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: invalid escape sequence '\m'
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: invalid escape sequence '\,'
  # Remove the CWD from sys.path while we load

In [ ]:
for i,t in enumerate(df.text):
    if '\m' in t:
        print(i,t)

7848 @iamdavebriggs see you in Manila on Sept  18! Check this out! One of my shots from Smash Project in Manila! \m/ http://t.co/SMxPUrwk
12383 @TheMetalCore Can't wait for them both! September is equally impressive with FFDP, Iron Maiden, Slayer, BMTH, and Parkway Drive! \m/
15805 Tomorrow is a big Metal day.  At 8:00am the new Iron Maiden tune premiers and, of course, new albums to purchase.  I love Heavy Metal! \m/
16328 Iron Maiden released their new video on the 14th of august \m/
16553 Check out the new Iron Maiden Video from their forthcoming album Book of Souls out September 4th! \m/ Listen to... http://t.co/pVDxR2iyQX
20152 Another great preview from the new IRON MAIDEN album THE BOOK OF SOULS \m/ OUT SEPTEMBER 4th http://t.co/sxrPyjyhMH
22131 Let's hope it's not!! Look for "The Book of Souls" from Iron Maiden This Friday September 4th!! \m/\m/ Listen to... http://t.co/R39jz4rmER
22323 @TonyBasilio Iron Maiden reference... Now you're talking! New album #BookOfSouls drops Frida

In [ ]:
df.text[1192]

'Lenny Kravitz in The Hunger Games.... Mmmm...May the odds be ever in our favor.'

In [ ]:
spacy_cleaner(df.text[1192])

'Lenny Kravitz in the Hunger Games mm May the odd be ever in our favor'

# Imbalanced Learning

The dataset isn't balanced where the neutral sentiment has 3 times more data than the negative and the positive class has around 2.4 times more data than the negative class

In [ ]:
df.sentiment.value_counts()

neutral     19466
positive    15754
negative     6485
Name: sentiment, dtype: int64

In [ ]:
#Original Imbalanced Data
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier

tvec = TfidfVectorizer(stop_words=None, max_features=100000, ngram_range=(1, 3))
lr = LogisticRegression(max_iter=1000)
rf = RandomForestClassifier(max_depth=2, random_state=0)
neigh = KNeighborsClassifier(n_neighbors=3)
gnb = GaussianNB()
pct = Perceptron(tol=1e-3, random_state=0)
sgd = SGDClassifier(loss="hinge", penalty="l2", max_iter=1000)

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_score, recall_score, f1_score

def lr_cv(splits, X, Y, pipeline, average_method):
    
    kfold = StratifiedKFold(n_splits=splits, shuffle=True, random_state=777)
    accuracy = []
    precision = []
    recall = []
    f1 = []
    for train, test in kfold.split(X, Y):
        lr_fit = pipeline.fit(X[train], Y[train])
        prediction = lr_fit.predict(X[test])
        scores = lr_fit.score(X[test],Y[test])
        
        accuracy.append(scores * 100)
        precision.append(precision_score(Y[test], prediction, average=average_method)*100)
        print('              negative    neutral     positive')
        print('precision:',precision_score(Y[test], prediction, average=None))
        recall.append(recall_score(Y[test], prediction, average=average_method)*100)
        print('recall:   ',recall_score(Y[test], prediction, average=None))
        f1.append(f1_score(Y[test], prediction, average=average_method)*100)
        print('f1 score: ',f1_score(Y[test], prediction, average=None))
        print('-'*50)

    print("accuracy: %.2f%% (+/- %.2f%%)" % (np.mean(accuracy), np.std(accuracy)))
    print("precision: %.2f%% (+/- %.2f%%)" % (np.mean(precision), np.std(precision)))
    print("recall: %.2f%% (+/- %.2f%%)" % (np.mean(recall), np.std(recall)))
    print("f1 score: %.2f%% (+/- %.2f%%)" % (np.mean(f1), np.std(f1)))

In [ ]:
from sklearn.pipeline import Pipeline
original_pipeline = Pipeline([
    ('vectorizer', tvec),
    ('classifier', lr)
])
lr_cv(5, df.clean_text, df.sentiment, original_pipeline, 'macro')

              negative    neutral     positive
precision: [0.63374486 0.65007445 0.71556166]
recall:    [0.35620663 0.78499872 0.66105998]
f1 score:  [0.45607108 0.71119386 0.68723194]
--------------------------------------------------
              negative    neutral     positive
precision: [0.63716814 0.6432319  0.71083506]
recall:    [0.33307633 0.78731056 0.65376071]
f1 score:  [0.43746835 0.70801571 0.68110431]
--------------------------------------------------
              negative    neutral     positive
precision: [0.62898936 0.64623838 0.71263563]
recall:    [0.36468774 0.78551246 0.64614408]
f1 score:  [0.46168863 0.70910145 0.67776298]
--------------------------------------------------
              negative    neutral     positive
precision: [0.63213811 0.65423365 0.72117687]
recall:    [0.36700077 0.78397123 0.66899397]
f1 score:  [0.46439024 0.71325076 0.69410603]
--------------------------------------------------
              negative    neutral     positive
precision

With data as it is without any resampling, we can see that the precision is higher than the recall.

## Oversampling


In [ ]:
#Random Over Sampler
from imblearn.pipeline import make_pipeline
from imblearn.over_sampling import ADASYN, SMOTE, RandomOverSampler

ROS_pipeline = make_pipeline(tvec, RandomOverSampler(random_state=777),lr )
SMOTE_pipeline_lr = make_pipeline(tvec, SMOTE(random_state=777),lr)
SMOTE_pipeline_rf = make_pipeline(tvec, SMOTE(random_state=777),rf)
SMOTE_pipeline_neigh = make_pipeline(tvec, SMOTE(random_state=777),neigh)
SMOTE_pipeline_gnb = make_pipeline(tvec, SMOTE(random_state=777),gnb)
SMOTE_pipeline_pct = make_pipeline(tvec, SMOTE(random_state=777),pct)
SMOTE_pipeline_sgd = make_pipeline(tvec, SMOTE(random_state=777),sgd)



In [ ]:
lr_cv(5, df.clean_text, df.sentiment, ROS_pipeline, 'macro')

              negative    neutral     positive
precision: [0.50431034 0.69983775 0.71215634]
recall:    [0.63145721 0.66478294 0.68232307]
f1 score:  [0.56076686 0.6818601  0.69692058]
--------------------------------------------------
              negative    neutral     positive
precision: [0.50473784 0.69218501 0.71061415]
recall:    [0.61603701 0.66889288 0.67565852]
f1 score:  [0.55486111 0.68033965 0.69269562]
--------------------------------------------------
              negative    neutral     positive
precision: [0.49114233 0.69041243 0.70538721]
recall:    [0.61989206 0.66221423 0.6648683 ]
f1 score:  [0.54805726 0.6760194  0.68452867]
--------------------------------------------------
              negative    neutral     positive
precision: [0.52075472 0.69913886 0.71630972]
recall:    [0.6383963  0.66735166 0.6899397 ]
f1 score:  [0.57360582 0.68287554 0.70287747]
--------------------------------------------------
              negative    neutral     positive
precision

In [ ]:
#SMOTE with Logistic Regression
lr_cv(5, df.clean_text, df.sentiment, SMOTE_pipeline_lr, 'macro')


              negative    neutral     positive
precision: [0.51315789 0.70013351 0.71333333]
recall:    [0.63145721 0.67351657 0.67914948]
f1 score:  [0.56619426 0.68656716 0.69582182]
--------------------------------------------------
              negative    neutral     positive
precision: [0.5094217  0.68792831 0.70711436]
recall:    [0.60447186 0.67043411 0.6750238 ]
f1 score:  [0.5528914  0.67906856 0.69069654]
--------------------------------------------------
              negative    neutral     positive
precision: [0.49876999 0.69119615 0.70550705]
recall:    [0.62528913 0.66349859 0.66677245]
f1 score:  [0.55490934 0.67706422 0.68559308]
--------------------------------------------------
              negative    neutral     positive
precision: [0.52753442 0.70350549 0.72022621]
recall:    [0.64996145 0.67531467 0.68708347]
f1 score:  [0.58238342 0.68912189 0.70326458]
--------------------------------------------------
              negative    neutral     positive
precision

In [ ]:
#SMOTE with Random Forest
lr_cv(5, df.clean_text, df.sentiment, SMOTE_pipeline_rf, 'macro')


              negative    neutral     positive
precision: [0.32413793 0.54243624 0.58752621]
recall:    [0.47108712 0.63370152 0.35576008]
f1 score:  [0.3840352  0.5845279  0.44317059]
--------------------------------------------------
              negative    neutral     positive
precision: [0.33388796 0.55281403 0.67870257]
recall:    [0.46414803 0.69637811 0.35195176]
f1 score:  [0.3883871  0.61634648 0.46353187]
--------------------------------------------------
              negative    neutral     positive
precision: [0.33405876 0.5827184  0.6342311 ]
recall:    [0.47340015 0.62882096 0.46334497]
f1 score:  [0.39170654 0.60489251 0.53548505]
--------------------------------------------------
              negative    neutral     positive
precision: [0.35930736 0.56484748 0.66246719]
recall:    [0.44795682 0.6992037  0.40050778]
f1 score:  [0.39876458 0.62488522 0.49920886]
--------------------------------------------------
              negative    neutral     positive
precision

In [ ]:
#SMOTE with KNN
lr_cv(5, df.clean_text, df.sentiment, SMOTE_pipeline_neigh, 'macro')


              negative    neutral     positive
precision: [0.48175182 0.47338689 0.66806723]
recall:    [0.05088666 0.9686617  0.05046017]
f1 score:  [0.09205021 0.63597268 0.09383299]
--------------------------------------------------
              negative    neutral     positive
precision: [0.20485981 0.63150074 0.58541033]
recall:    [0.84502699 0.21834061 0.30561726]
f1 score:  [0.32977283 0.32448941 0.40158465]
--------------------------------------------------
              negative    neutral     positive
precision: [0.36585366 0.47397606 0.73140496]
recall:    [0.0462606  0.96609299 0.05617264]
f1 score:  [0.08213552 0.6359486  0.10433245]
--------------------------------------------------
              negative    neutral     positive
precision: [0.20595855 0.63908702 0.60521173]
recall:    [0.85813416 0.23015669 0.29482704]
f1 score:  [0.33218923 0.33843248 0.39650021]
--------------------------------------------------
              negative    neutral     positive
precision

In [ ]:
#SMOTE with Perceptron
lr_cv(5, df.clean_text, df.sentiment, SMOTE_pipeline_pct, 'macro')


              negative    neutral     positive
precision: [0.5122549  0.66936354 0.64153711]
recall:    [0.48342328 0.63755459 0.69406538]
f1 score:  [0.49742166 0.65307196 0.66676829]
--------------------------------------------------
              negative    neutral     positive
precision: [0.55304519 0.64733465 0.64445124]
recall:    [0.43407864 0.67377344 0.66899397]
f1 score:  [0.48639309 0.66028949 0.6564933 ]
--------------------------------------------------
              negative    neutral     positive
precision: [0.52522523 0.64977916 0.63867534]
recall:    [0.44949884 0.64243514 0.68549667]
f1 score:  [0.48442044 0.64608628 0.66125823]
--------------------------------------------------
              negative    neutral     positive
precision: [0.53799655 0.65195369 0.67961804]
recall:    [0.48033924 0.69432314 0.65503015]
f1 score:  [0.50753564 0.6724717  0.66709761]
--------------------------------------------------
              negative    neutral     positive
precision

In [ ]:
#SMOTE with SGD
lr_cv(5, df.clean_text, df.sentiment, SMOTE_pipeline_sgd, 'macro')


              negative    neutral     positive
precision: [0.44676894 0.71232877 0.71428571]
recall:    [0.74094063 0.60107886 0.6585211 ]
f1 score:  [0.55742459 0.6519922  0.68527081]
--------------------------------------------------
              negative    neutral     positive
precision: [0.4393869  0.70837224 0.70521008]
recall:    [0.72937548 0.5846391  0.66582037]
f1 score:  [0.5484058  0.64058542 0.6849494 ]
--------------------------------------------------
              negative    neutral     positive
precision: [0.43523551 0.71763216 0.7067922 ]
recall:    [0.74094063 0.58232725 0.66708981]
f1 score:  [0.54835949 0.64293817 0.68636735]
--------------------------------------------------
              negative    neutral     positive
precision: [0.45540797 0.71630335 0.71337366]
recall:    [0.74016962 0.59928076 0.67375436]
f1 score:  [0.56387665 0.65258741 0.6929982 ]
--------------------------------------------------
              negative    neutral     positive
precision

## Downsampling

In [ ]:
from imblearn.under_sampling import NearMiss, RandomUnderSampler
from imblearn import under_sampling as us

RUS_pipeline = make_pipeline(tvec, RandomUnderSampler(random_state=777),lr)
NM1_pipeline = make_pipeline(tvec, NearMiss(sampling_strategy='not minority', version = 1),lr)
NM2_pipeline = make_pipeline(tvec, NearMiss(sampling_strategy='not minority', version = 2),lr)
NM3_pipeline = make_pipeline(tvec, NearMiss(sampling_strategy='not minority', version = 3, n_neighbors_ver3=4),lr)

### RandomUnderSampler


In [ ]:
lr_cv(5, df.clean_text, df.sentiment, RUS_pipeline, 'macro')


              negative    neutral     positive
precision: [0.43359197 0.7091081  0.69398724]
recall:    [0.73245952 0.57796044 0.65566487]
f1 score:  [0.54472477 0.63685253 0.67428198]
--------------------------------------------------
              negative    neutral     positive
precision: [0.43256881 0.69725064 0.68282229]
recall:    [0.72706245 0.56023632 0.65725167]
f1 score:  [0.54242163 0.62127902 0.66979301]
--------------------------------------------------
              negative    neutral     positive
precision: [0.42041908 0.69575089 0.67794362]
recall:    [0.72706245 0.55098895 0.64868296]
f1 score:  [0.53276836 0.6149656  0.66299059]
--------------------------------------------------
              negative    neutral     positive
precision: [0.44020475 0.70205371 0.68913115]
recall:    [0.72937548 0.57076805 0.66201206]
f1 score:  [0.54904237 0.62964012 0.67529945]
--------------------------------------------------
              negative    neutral     positive
precision

## Near Miss

### Near Miss 1

In [ ]:
lr_cv(5, df.clean_text, df.sentiment, NM1_pipeline, 'macro')


              negative    neutral     positive
precision: [0.40577778 0.69964912 0.65751311]
recall:    [0.70393215 0.51220139 0.67629324]
f1 score:  [0.51480124 0.59142815 0.66677096]
--------------------------------------------------
              negative    neutral     positive
precision: [0.41160347 0.69470828 0.66853933]
recall:    [0.69468003 0.52607244 0.6797842 ]
f1 score:  [0.51692484 0.59874287 0.67411487]
--------------------------------------------------
              negative    neutral     positive
precision: [0.40605521 0.70614797 0.66542289]
recall:    [0.70316114 0.52221937 0.67914948]
f1 score:  [0.51481795 0.60041347 0.67221611]
--------------------------------------------------
              negative    neutral     positive
precision: [0.41906475 0.70777664 0.67010632]
recall:    [0.71858134 0.53069612 0.68010156]
f1 score:  [0.52939506 0.60657663 0.67506694]
--------------------------------------------------
              negative    neutral     positive
precision

## Near Miss 2

In [ ]:
lr_cv(5, df.clean_text, df.sentiment, NM2_pipeline, 'macro')


              negative    neutral     positive
precision: [0.35426166 0.6952487  0.68919886]
recall:    [0.77872012 0.48111996 0.61155189]
f1 score:  [0.48698168 0.56869592 0.64805784]
--------------------------------------------------
              negative    neutral     positive
precision: [0.36727941 0.70454545 0.68855859]
recall:    [0.77023901 0.49370665 0.63218026]
f1 score:  [0.49738611 0.58057695 0.65916612]
--------------------------------------------------
              negative    neutral     positive
precision: [0.36512559 0.69652454 0.69389939]
recall:    [0.77332305 0.49935782 0.61726436]
f1 score:  [0.49604352 0.58168761 0.65334229]
--------------------------------------------------
              negative    neutral     positive
precision: [0.37924456 0.7053701  0.68893387]
recall:    [0.76638396 0.49935782 0.64804824]
f1 score:  [0.50740174 0.58474959 0.6678659 ]
--------------------------------------------------
              negative    neutral     positive
precision

### Near Miss 3

In [ ]:
lr_cv(5, df.clean_text, df.sentiment, NM3_pipeline, 'macro')


              negative    neutral     positive
precision: [0.45563674 0.69061489 0.64557721]
recall:    [0.67309175 0.54816337 0.68327515]
f1 score:  [0.54341737 0.61119863 0.66389146]
--------------------------------------------------
              negative    neutral     positive
precision: [0.44405768 0.68429287 0.66145501]
recall:    [0.68851195 0.56177755 0.65788639]
f1 score:  [0.53990326 0.61701227 0.65966587]
--------------------------------------------------
              negative    neutral     positive
precision: [0.44984955 0.69017713 0.64404688]
recall:    [0.69159599 0.55047521 0.66264678]
f1 score:  [0.54512306 0.6124607  0.65321445]
--------------------------------------------------
              negative    neutral     positive
precision: [0.4449926  0.69025706 0.66708821]
recall:    [0.69545104 0.55869509 0.66962869]
f1 score:  [0.54271961 0.61754685 0.66835603]
--------------------------------------------------
              negative    neutral     positive
precision

In [ ]:
from collections import Counter

nm3 = NearMiss(sampling_strategy='not minority', version=3, n_neighbors_ver3=4)
tvec = TfidfVectorizer(stop_words=None, max_features=100000, ngram_range=(1, 3))
df_tfidf = tvec.fit_transform(df.clean_text)
X_res, y_res = nm3.fit_resample(df_tfidf, df.sentiment)
print('Distribution before NearMiss-3: {}'.format(Counter(df.sentiment)))
print('Distribution after NearMiss-3: {}'.format(Counter(y_res)))

Distribution before NearMiss-3: Counter({'neutral': 19466, 'positive': 15754, 'negative': 6485})
Distribution after NearMiss-3: Counter({'negative': 6485, 'neutral': 6485, 'positive': 6485})


In [ ]:
from collections import Counter

nm2 = NearMiss(sampling_strategy='not minority', version=2, n_neighbors_ver3=4)
tvec = TfidfVectorizer(stop_words=None, max_features=100000, ngram_range=(1, 3))
df_tfidf = tvec.fit_transform(df.clean_text)
X_res, y_res = nm2.fit_resample(df_tfidf, df.sentiment)
print('Distribution before NearMiss-2: {}'.format(Counter(df.sentiment)))
print('Distribution after NearMiss-2: {}'.format(Counter(y_res)))

Distribution before NearMiss-2: Counter({'neutral': 19466, 'positive': 15754, 'negative': 6485})
Distribution after NearMiss-2: Counter({'negative': 6485, 'neutral': 6485, 'positive': 6485})


In [ ]:
from collections import Counter

nm1 = NearMiss(sampling_strategy='not minority', version=1, n_neighbors_ver3=4)
tvec = TfidfVectorizer(stop_words=None, max_features=100000, ngram_range=(1, 3))
df_tfidf = tvec.fit_transform(df.clean_text)
X_res, y_res = nm1.fit_resample(df_tfidf, df.sentiment)
print('Distribution before NearMiss-1: {}'.format(Counter(df.sentiment)))
print('Distribution after NearMiss-1: {}'.format(Counter(y_res)))

Distribution before NearMiss-1: Counter({'neutral': 19466, 'positive': 15754, 'negative': 6485})
Distribution after NearMiss-1: Counter({'negative': 6485, 'neutral': 6485, 'positive': 6485})
